



# Unscented Kalman Filters
* Use sigma points to approximate Prob dist to deal with non-linear process and measurement functions
    * Often better approximation than EKF
    * Don't need to calculate Jacobian

### Lesson outline:
* More sophisticated process (model) that can estimate the turn rate of a vehicle
* Go through how the Unscented Kalman Filter deals with this non-linear process model
* Implement the UKF in C++.

## CTRV model (Constant Turn Rate and Velocity Magnitude Model)
* Relaxes assumption of vehicle's constant velocity 
    * When vehicles are turning, they move slower. So EKF's predicted position would be outside of the circular path of the car.
    
![](images/7-3.png)

* Describe velocity using Speed (v) and yaw (psi) angle
    * Vehicle moving in a straight line has a yaw rate (psi dot) of zero.
* Prediction: find f, where $x_(k+1) = f(x_k, v_k)$.
* Approach: Describe the change rates of the state x.
* Assumes constant turn rate (yaw dot) and constant velocity.

![](images/7-6.png)
![](images/7-7.png)

### Process noise
![](images/7-8.png)

Uncertainty nu_k
Two noise processes:
* v: Longitudinal acceleration noise, changes at each timestep k
* yaw

How does the noise vector influence the process model?

Sign of yaw acceleration noise:
![](images/7-9.png)
![](images/7-10.png)
![](images/7-11.png)
![](images/7-12.png)
![](images/7-13.png)
![](images/7-14.png)


## Unscented Kalman Filter
* Predict and update steps as before

### Unscented transformation

* Recap: issue with nonlinear process or measurement models is that predicted state distribution is usually not normal.
    * UKF goes on with normal distribution with same mean and variance as predicted distribution (an approximation
* Solve using sigma points (representatives of the whole distribution)
    * Transform individual points of the state space thorugh the non-linear function.
    * Calculate mean and variance of the sigma points.

Steps:
* Choose sigma points
* Predict sigma points (insert into process function)
* Calculate mean and variance


![](images/7-15.png)
* Posterior state x_k|k, posterior covariance matrix Pk|k.
* Consider two dimensions of the state first (px, py).

Sigma points
* Mean of state
* Two points per state dimension.

![](images/7-16.png)
* Lambda (choose): distance of state dim sigma points from the mean state
* Square root of matrix Pk|k: directions of the sigma points relative to the mean.
* xk|k: first sigma point (mean)
* first vector points 

![](images/7-17.png)
![](images/7-18.png)
![](images/7-19.png)

#### Generate Sigma Points

### UKF Augmentation
* Previously generated sigma points: can put sigma points into process function.
* But Process noise nu_k is also non-linear (need to deal with this). UKF has a way to handle this.

* Aside: Difference expressions of nu_k in literature: 
    * two-dim: independent noise processes.
    * five-dim: each row of vector tells influence of process noise on each element of the state
    

* Covariance noise for process noise Q.
    * ![](images/7-18.png)
    * covar=0 because longitudinal acceleration noise and yaw acceleration noise are considered to be independent.
* Represent uncertainty of Q with sigma points using augmentation
    * Augmented state: Add noise vector to state vector
    * Additional sigma points represent uncertainty caused by process noise 
    * Build 7x7 augmented covariance matrix P
    * ![](images/7-20.png)

#### Augmentation 
![](images/7-21.png)

![](images/7-22.png)

### Predicting sigma points
* Insert sigma points into model (chi_{k+1|k}).
* Input to process model is 7-dim vector, output is 5-dim (without 2-dim noise).

![](images/7-23.png)
![](images/7-24.png)
Code notes:
* Xsig_aug: evey column is a sigma point.

### Predict Mean and Covariance (of predicted state)

![](images/7-25.png)
![](images/7-26.png)

### Measurement Prediction

![](images/7-27.png)
![](images/7-28.png)
![](images/7-29.png)
![](images/7-30.png)

* Transform predicted state into measurement state using the measurement model h.
    * Problem: Need to transform dist through non-linear distribution. So apply same process as before.
* Shortcuts: 
    * Re-use sigma points from previous step.
        * Calculate mean z and predicted covariance S.
    * Skip augmentation step (process noise had non-linear effect on state, but measurement noise has pure additive effect).
        * Instead. [insert]


### Update state
![](images/7-31.png)
![](images/7-32.png)
![](images/7-33.png)
![](images/7-34.png)
![](images/7-35.png)

### Parameters and Consistency

How to choose noise parameters and how to evaluate them.
* Can apply to any Bayesian filters.

* Radar measurement noise: describes how precise the sensor is. Get from sensor manual.
    * ![](images/7-36.png)
* Process noise covariance: Strong approximation by assuming white process noise.
    * If it were so, drivers would constantly switch between stepping on the gas and on the brake. [More detailed translation?]
    * Rule of thumb: Estimate maximum expected acceleration, e.g. 6ms^(-2) for urban cars. Then choose half that max acceleration as the SD of the acceleration noise.
    * If application needs to react quickly to changes, increase process noise variance. If it needs to give smooth predictions, decrease process noise variance.
* Evaluating noise parameters: Run a consistency check.
    * Compare predicted measurement mean and covariance and actual prediction. e.g. systematically underestimate (or overestimate) precision of prediction.
* Do this using Normalised Innovation Squared
![](images/7-37.png)

    * Innovation = diff between predicted and actual measurements. 
    * normalised: put it in relation to covariance matrix S.
* NIS: Scalar number that follows a chi-squared distribution. 
    * df (degrees of freedom) here are the dimensions of our measurement space.
* Implementing: Plot 95% line and calculate and plot the NIS values.

Underestimate uncertainty:
![](images/7-38.png)
Overestimate uncertainty:
![](images/7-39.png)

NIS test does not tell you where mistake comes from, but gives you indications.

### Comments (rename section)
![](images/7-40.png)
Green line: noisy position measurements.

Previous project: have problems following the turn. UKF follows the turn better and provides smooth predictions (lower process noise).

Radar NIS:
![](images/7-41.png)
Laser NIS:

Consistent -> realistic covariance matrix.

Velocity estimate converges faster
![](images/7-42.png)

Advantages: UKF estimates orientation of bicycle precisely even though it is not directly measured by any sensor directly.
Important because of vehicles changing lanes.
* Can take noisy measurement data as input, provide smooth estimation.
* Provide estimation of orientation (yaw rate) of vehicles using sensors that can’t directly measure these things
* Info on how precise result is because provides covariance matrix. -> can check if covariance matrix is realistic if UKF passes consistency check.

Unscented because they felt EKF stank.


Outro:


- Sensors record the data
- ...
Extensions:
* What if there are many objects to track? Find out which measurements belong to which vehicles.
